In [1]:
%pylab --no-import-all
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
from scipy.io import wavfile
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
gridpath = '../model/res/super_concurrent_170419_152612/'
run_paths = sorted([run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)])

gridanalysis = utils.GridAnalyser(run_paths)

In [8]:
with open(join(gridpath, 'desc.md')) as f:
    display(Markdown(f.read()))

# Run description

## Goals

- Understand the impact replays and number of models


## What have changed compared to previous runs?

-

## Expected Results (Success condition)

- More replay should lead in less diversity
- More replay should lead in increase in performance
- More models should lead in more diversity
- More models should lead in more unlearning


# Results


## Score

- The score goes always way below the score of the synthesized song with
  BA algorithm. Yet, the BA algorithm capture better the song.
- The score seems to stagnate starting from the 20th day.

## replays

- More replays lead to more gesture. There is no convergence on the right
  number of gesture. more is always better.
- There is no need to have a lot of gesture to capture the temporality.
  Might be due to the behaviour of subzero alpha.
- Slightly faster convergence with plenty of replays.


## Number of models

- There is very little diversity even with plenty of models
- More replays have a negative impact on this diversity.
- Slower convergence with a lot of models because less training per model
- More models helps unlearning: There is plenty of run where for a few models,
  the score changes a lot in a negative way between evening and morning. Plenty
  of replays has a negative impact on this behaviour (expected)


# Remarks

- Gestures were fixed and with fixed size sounds.
- Genetic algorithm was pretty dumb one
- There was a lot of training, even if it underestimated by a factor of two the
  amount of syllables recorded by *Derégnaucourt et al.*


In [6]:
vbox = widgets.VBox()

tbs = []
for options in gridanalysis.options_list:
    tb = widgets.ToggleButtons()
    tb.options = list(options)
    display(tb)
    tbs.append(tb)

def find_id(value):
    for i in range(len(gridanalysis.run_paths)):
        if gridanalysis.conf[i]['name'] == '+'.join([tb.value for tb in tbs]):
            slider.value = i
            break

slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: gridanalysis.show(x['new'], vbox), names='value')
display(slider)
display(vbox)

gridanalysis.show(slider.value, vbox)

for tb in tbs:
    tb.observe(lambda x: find_id(x['new']), names='value')

In [7]:
display(slider)